## CMF 1
Institut für Musikinformatik und Musikwissenschaft – Wintersemester 2025–26
### Woche 06 – Vorlesung

### Computergestützte Fugenanalyse II

In [21]:
import music21 as m21
import numpy as np

1) Implementierung der Levenshtein-Distanz für Zeichenketten

In [48]:
def levenshtein_distance(list1, list2):
    M = len(list1) + 1
    N = len(list2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Wort-Präfixen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):
            if (i == 0 and j == 0):
                D[i, j] = 0

            elif (i == 0 or j == 0):
                D[i, j] = 1000 

            elif list1[i-1] == list2[j-1]:
                D[i, j] = D[i-1, j-1]

            else:
                min_value = min(D[i-1, j-1], D[i-1, j], D[i, j-1])
                D[i, j] = 1 + min_value    

    return D[i, j]

2) Implementierung einer Funktion, um Notenlisten in Intervalllisten umzuwandeln

In [61]:
def notes_to_intervals(note_list=[m21.note.Note("C4"), m21.note.Note("E4"), m21.note.Note("G4")]):
    interval_dictionary = {0: 1, 1:2, 2:2, 3:3, 4:3, 5:4, 6:4.5, 7:5, 8:6, 9:6, 10:7, 11:7, 12:8, 13:9, 14:9, 15:10, 16:10, 17:11,
                                -1:-2, -2:-2, -3:-3, -4:-3, -5:-4, -6:-4.5, -7:-5, -8:-6, -9:-6, -10:-7, -11:-7, -12:-8, -13:-9, -14:-9, -17:-11}

    num_notes = len(note_list)
    interval_list = []

    for i in range(num_notes - 1):

        current_note = note_list[i].pitch.midi
        next_note = note_list[i+1].pitch.midi

        current_interval = next_note - current_note    
        interval_list.append(current_interval)
    diatonic_interval_list = [interval_dictionary[interval] for interval in interval_list]

    return diatonic_interval_list

3) Ein Dictionary...

In [43]:
fugeScore = m21.converter.parse("bach_fugue_cminor.krn")

In [64]:
altoNote = list(fugeScore.parts[1].flatten().getElementsByClass(m21.note.Note))
altoInvervals = notes_to_intervals(altoNote)

sopranoNotes = list(fugeScore.parts[0].flatten().getElementsByClass(m21.note.Note))
sopranoIntervals = notes_to_intervals(sopranoNotes)

bassNotes = list(fugeScore.parts[2].flatten().getElementsByClass(m21.note.Note))
bassIntervals = notes_to_intervals(bassNotes)

In [65]:
fugeTheme = altoInvervals[:19]

In [66]:
def subFinder(themeIntervals, voiceIntervals):
    lenSubject = len(themeIntervals)
    subjectIndices = [] #where theme was found
    distances = [] #distance to theme
    threshhold = 4
    for i in range(len(voiceIntervals)):
        currentDistance = levenshtein_distance(themeIntervals, voiceIntervals[i:i+lenSubject])
        if currentDistance <= threshhold:
            subjectIndices.append(i)
            distances.append(float(currentDistance))
    return subjectIndices, distances

In [67]:
print(subFinder(fugeTheme, altoInvervals))

([0, 120], [0.0, 2.0])


In [68]:
print(subFinder(fugeTheme, sopranoIntervals))

([0, 72, 161, 247], [2.0, 0.0, 0.0, 2.0])


In [69]:
print(subFinder(fugeTheme, bassIntervals))

([0, 211], [0.0, 0.0])
